<a href="https://colab.research.google.com/github/lognat0704/TopGun/blob/main/7_Pytorch_tips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import torch 
from torch import nn

In [2]:
start_time = time.time()
for _ in range(100):
  cpu_tensor = torch.ones((1000,64,64))
  gpu_tensor = cpu_tensor.cuda()

print('Total time: {:.3f}s'.format(time.time()-start_time))

Total time: 12.935s
